In [25]:
from __future__ import print_function
from yql.api.v1.client import YqlClient

In [ ]:
!git clone https://github.yandex-team.ru/edvls/statbox-abt-metrics.git
from statbox_abt_metrics.api import run
from statbox_abt_metrics.metrics.metrics_group import (
    metrics_group, YTMetricsGroup
)

In [ ]:
import statbox_abt_metrics

In [36]:
users_table = "//home/mobilesearch/personal_homes/bobsmirnov/experiments-21378/glued"
start_date = "2018-06-14"
end_date = "2018-07-09"
control_id = "control"
experiment_id = "take-over"

YQL_TOKEN = "XXXXX" # Open https://yql.yandex-team.ru/?settings_mode=token in browser to get your token
tmp_sessions_floder = "tmp/edvls_sessions_copy" 

In [32]:
yql_request = '''USE hahn;
PRAGMA yt.InferSchema;
PRAGMA SimpleColumns;

$tasks_folder = "home/voice/dialog/sessions";
$results_folder = "{tmp_floder}";

DEFINE ACTION $process_task($input_path) AS
    $output_path = String::ReplaceAll($input_path, $tasks_folder, $results_folder);
    
    INSERT INTO $output_path WITH TRUNCATE
    SELECT 
        YQL::Append(Yson::ConvertToStringList(sessions.testids), users.exp) AS testids,
        sessions.* WITHOUT sessions.testids
    FROM $input_path AS sessions 
    JOIN [{users_table}] AS users
    ON  String::SplitToList(sessions.uuid, "/"){{1}} == users.uuid
    --WHERE users.exp_start <= sessions.fielddate 
    ;
END DEFINE;

$tasks = (
    SELECT LIST(Path)
    FROM FOLDER($tasks_folder)
    WHERE Type == "table"
    AND Path >= $tasks_folder || "/{start_date}"
    AND Path <= $tasks_folder || "/{end_date}"
);

EVALUATE FOR $task IN $tasks ?? ListCreate(TypeOf($tasks)) DO $process_task($task);
'''.format(tmp_floder=tmp_sessions_floder, start_date=start_date, end_date=end_date, users_table=users_table)

In [33]:
yql.__dict__

{'__builtins__': {'ArithmeticError': ArithmeticError,
  'AssertionError': AssertionError,
  'AttributeError': AttributeError,
  'BaseException': BaseException,
  'BufferError': BufferError,
  'BytesWarning': BytesWarning,
  'DeprecationWarning': DeprecationWarning,
  'EOFError': EOFError,
  'Ellipsis': Ellipsis,
  'EnvironmentError': EnvironmentError,
  'Exception': Exception,
  'False': False,
  'FloatingPointError': FloatingPointError,
  'FutureWarning': FutureWarning,
  'GeneratorExit': GeneratorExit,
  'IOError': IOError,
  'ImportError': ImportError,
  'ImportWarning': ImportWarning,
  'IndentationError': IndentationError,
  'IndexError': IndexError,
  'KeyError': KeyError,
  'KeyboardInterrupt': KeyboardInterrupt,
  'LookupError': LookupError,
  'MemoryError': MemoryError,
  'NameError': NameError,
  'None': None,
  'NotImplemented': NotImplemented,
  'NotImplementedError': NotImplementedError,
  'OSError': OSError,
  'OverflowError': OverflowError,
  'PendingDeprecationWarning':

In [ ]:
client = YqlClient(token=YQL_TOKEN)
request = client.query(yql_request)
request.run()

In [38]:
SESSION_LOGS = tmp_sessions_floder

In [39]:
from statbox_abt_metrics.metrics.alice.basic_scenarios_metrics import *

@metrics_group
class BasicScenariosMetricsModified(YTMetricsGroup):
    reducer = IntentMetrics()

    name = "basic_scenarios_metrics"

    ru_name = "Базовые метрики сценариев"

    en_name = "Basic scenarios metrics"

    def build_meta(self):
        return make_metrics_meta()

    def build_stream(self, job):
        return job \
            .log(SESSION_LOGS) \
            .project(
                qe.all(),
                unfold_experiments(self.config.experiments),
                user_id='uuid'
            ) \
            .call(
                unfold_datetime,
                self.config,
                qe.log_field('fielddate')
            ) \
            .groupby('datetime', 'user_id', 'exp_id') \
            .reduce(self.reducer)

In [40]:
from statbox_abt_metrics.metrics.alice.basic_metrics import *

@metrics_group
class BasicMetricsModified(YTMetricsGroup):
    name = "basic_metrics"
    ru_name = "Базовые метрики"
    en_name = "Basic metrics"

    reducer = None

    def init(self):
        self.reducer = BasicMetricsCounter()

    def build_meta(self):
        return self.reducer.meta

    def build_stream(self, job):
        return job \
            .log(SESSION_LOGS) \
            .project(
                qe.all(),
                unfold_experiments(self.config.experiments),
                user_id='uuid'
            ) \
            .call(
                unfold_datetime,
                self.config,
                qe.log_field('fielddate')
            ) \
            .groupby('datetime', 'user_id', 'exp_id') \
            .reduce(self.reducer)

In [41]:
from statbox_abt_metrics.metrics.alice.session_metrics import *

@metrics_group
class SessionMetricsModified(YTMetricsGroup):
    name = "session_metrics"
    ru_name = "Сессионные метрики"
    en_name = "Sessions metrics"

    reducer = None

    def init(self):
        self.reducer = AliceSessionsCounterReduce()

    def build_meta(self):
        return self.reducer.meta

    def build_stream(self, job):
        return job.log(
            SESSION_LOGS
        ).project(
            qe.all(),
            unfold_experiments(self.config.experiments),
            user_id='uuid',
        ).groupby(
            'fielddate', 'user_id', 'exp_id',
        ).sort(
            'session_id',
        ).reduce(
            unfold_sessions,
        ).call(
            unfold_datetime,
            self.config,
            qe.log_field('fielddate'),
        ).groupby(
            'datetime', 'user_id', 'exp_id',
        ).sort(
            'timestamp',
        ).reduce(
            self.reducer,
        )

In [ ]:
result = run(
    service='transport',
    metric_groups=[BasicMetricsModified, SessionMetricsModified, BasicScenariosMetricsModified],
    experiments=[control_id, experiment_id],
    start_date=start_date,
    end_date=end_date
)

In [ ]:
result